# Cleaning data
Main takeaways:
 - In this case Z-Score provided a real advantage, resulting in the best model so far
 - Higher learning rate is preferred with lower feature values
 - MinMax didn't provide an edge over the basic dataset
 
 Known limitations of this notebook:
 - no test-set-validation
 - no normalization

In [1]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from helper import charts, lib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
# Import the dataset.
training_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")

#scale the label
training_df["median_house_value"] /= 1000.0

#set the label name early
my_label="median_house_value" # the median value of a house on a specific city block.

### Removing outlying values, creating 4 versions: 
 - outliers removed from both housing_median_age and median_house_value
 - outliers removed from only one of them
 - no outliers removed

In [3]:
#names will be hma, mhv, both, none

#because they occur a lot
hma = "housing_median_age"
mhv = "median_house_value"

hma_max = training_df[hma].max()
_hma_drop = training_df.loc[training_df[hma] == hma_max].index
print("Dataset contains", len(_hma_drop), "elements in the '",hma,"' column with the value", hma_max, "(max value)")

mhv_max = training_df[mhv].max()
_mhv_drop = training_df.loc[training_df[mhv] == mhv_max].index
print("Dataset contains", len(_mhv_drop), "elements in the '",mhv,"' column with the value", mhv_max, "(max value)")



Dataset contains 1052 elements in the ' housing_median_age ' column with the value 52.0 (max value)
Dataset contains 814 elements in the ' median_house_value ' column with the value 500.001 (max value)


In [4]:
hma_df = training_df.drop(_hma_drop)
mhv_df = training_df.drop(_mhv_drop)
both_df = hma_df.drop(_mhv_drop, errors='ignore')
none_df = training_df.copy(deep=True)

In [5]:
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(units=1, input_shape=(6,)))
 
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model        


def train_model(model, training_df, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""
  history = model.fit(
    x=training_df[feature],
    y=training_df[label],
    batch_size=batch_size,
    #validation_data=(training_df[feature], training_df[label]), #the weights shouldn't be influenced by the model, but the overall accurarcy should be measured against all datapoints
    epochs=epochs)

  return pd.DataFrame(history.history)

In [6]:
# Specify the feature and the label.
my_feature = ["longitude", "latitude", "total_rooms", "total_bedrooms", "population", "households"]

In [7]:
# Hyperparameters for none
learning_rate = 0.01
epoch_count = 20
batch_size = 64

# Build model
none_model = build_model(learning_rate)
none_history = train_model(none_model, none_df, my_feature, my_label, epoch_count, batch_size)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/20
266/266 [==============================] - 3s 11ms/step - loss: 177553.6875 - root_mean_squared_error: 421.3712 - val_loss: 78242.4375 - val_root_mean_squared_error: 279.7185
Epoch 2/20
266/266 [==============================] - 2s 9ms/step - loss: 45846.2344 - root_mean_squared_error: 214.1173 - val_loss: 28870.8379 - val_root_mean_squared_error: 169.9142
Epoch 3/20
266/266 [==============================] - 2s 8ms/step - loss: 22083.2949 - root_mean_squared_error: 148.6045 - val_loss: 16413.1055 - val_root_mean_squared_error: 128.1136
Epoch 4/20
266/266 [==============================] - 2s 8ms/step - loss: 14382.7139 - root_mean_squared_error: 119.9280 - val_loss: 12976.9707 - val_root_mean_squared_error: 113.9165
Epoch 5/20
266/266 [==============================] - 2s 8ms/step - loss: 12501.8184 - root_mean_squared_error: 111.8115 - val_loss: 12229.6191 - val_root_mean_squared_error: 110.58

In [8]:
# Hyperparameters for mhv
learning_rate = 0.01
epoch_count = 20
batch_size = 64

# Build model
mhv_model = build_model(learning_rate)
mhv_history = train_model(mhv_model, mhv_df, my_feature, my_label, epoch_count, batch_size)

Epoch 1/20
253/253 [==============================] - 2s 9ms/step - loss: 474432.2812 - root_mean_squared_error: 688.7905 - val_loss: 20219.5918 - val_root_mean_squared_error: 142.1956
Epoch 2/20
253/253 [==============================] - 2s 8ms/step - loss: 17157.9414 - root_mean_squared_error: 130.9883 - val_loss: 14169.0605 - val_root_mean_squared_error: 119.0339
Epoch 3/20
253/253 [==============================] - 2s 8ms/step - loss: 12526.7012 - root_mean_squared_error: 111.9227 - val_loss: 11092.9180 - val_root_mean_squared_error: 105.3229
Epoch 4/20
253/253 [==============================] - 2s 8ms/step - loss: 10413.9014 - root_mean_squared_error: 102.0485 - val_loss: 9753.4297 - val_root_mean_squared_error: 98.7595
Epoch 5/20
253/253 [==============================] - 2s 8ms/step - loss: 9505.0469 - root_mean_squared_error: 97.4938 - val_loss: 9196.6914 - val_root_mean_squared_error: 95.8994
Epoch 6/20
253/253 [==============================] - 2s 8ms/step - loss: 9072.0352 -

In [9]:
# Hyperparameters for HMA
learning_rate = 0.01
epoch_count = 20
batch_size = 64

# Build model
hma_model = build_model(learning_rate)
hma_history = train_model(hma_model, hma_df, my_feature, my_label, epoch_count, batch_size)

Epoch 1/20
250/250 [==============================] - 2s 8ms/step - loss: 83857.7344 - root_mean_squared_error: 289.5820 - val_loss: 15571.4951 - val_root_mean_squared_error: 124.7858
Epoch 2/20
250/250 [==============================] - 2s 9ms/step - loss: 12402.3535 - root_mean_squared_error: 111.3659 - val_loss: 11321.9463 - val_root_mean_squared_error: 106.4046
Epoch 3/20
250/250 [==============================] - 2s 8ms/step - loss: 11461.1719 - root_mean_squared_error: 107.0569 - val_loss: 11257.7461 - val_root_mean_squared_error: 106.1025
Epoch 4/20
250/250 [==============================] - 2s 8ms/step - loss: 11399.2686 - root_mean_squared_error: 106.7674 - val_loss: 11239.9521 - val_root_mean_squared_error: 106.0186
Epoch 5/20
250/250 [==============================] - 2s 8ms/step - loss: 11511.9902 - root_mean_squared_error: 107.2939 - val_loss: 11215.9219 - val_root_mean_squared_error: 105.9053
Epoch 6/20
250/250 [==============================] - 2s 8ms/step - loss: 11619.

KeyboardInterrupt: 

In [ ]:
# Hyperparameters for both
learning_rate = 0.01
epoch_count = 20
batch_size = 64

# Build model
both_model = build_model(learning_rate)
both_history = train_model(both_model, both_df, my_feature, my_label, epoch_count, batch_size)

: 

In [ ]:
#examining history
rmse = "root_mean_squared_error"

none_key = "Original"
mhv_key = "Mean House Value cleaned up"
hma_key = "Housing Median Age cleaned up"
both_key = "Both cleaned up"

keys = [none_key, mhv_key, hma_key, both_key]

together_df = pd.DataFrame({
    none_key : none_history[rmse],
    mhv_key : mhv_history[rmse],
    hma_key : hma_history[rmse],
    both_key : both_history[rmse]
})

: 

In [ ]:
#plotting basic rmse history of the three different methods
charts.plot_training_losses(losses=keys, df=together_df)

: 

In [ ]:
print("No cleaning metrics:")
_ = none_model.evaluate(training_df[my_feature], training_df[my_label])

print("MHV cleaned metrics:")
_ = mhv_model.evaluate(training_df[my_feature], training_df[my_label])

print("HMA cleaned metrics:")
_ = hma_model.evaluate(training_df[my_feature], training_df[my_label])

print("Both cleaned metrics:")
_ = both_model.evaluate(training_df[my_feature], training_df[my_label])

: 

In [ ]:
#sampling data
sample_normal = lib.create_inference_sample(training_df, my_feature, basic_model)
sample_z = lib.create_inference_sample(z_df, my_feature, z_model)
sample_minmax = lib.create_inference_sample(minmax_df, my_feature, minmax_model)

: 

In [ ]:
charts.plot_training_test_validat_accuracy(
    dfs=[sample_normal, sample_z, sample_minmax],
    label_key=my_label,
    titles=keys
)

: 

In [ ]:
#weights per features
weights = pd.DataFrame.from_dict({
    "feature" : my_feature,
    basic_key : basic_model.get_weights()[0].ravel(),
    z_key : z_model.get_weights()[0].ravel(),
    minmax_key : minmax_model.get_weights()[0].ravel(),
})

weights

: 